# Description
Add to the queueing simulator you have already developed (lab 1) some routines to:

a) detect the end of transient in an automated way (write a short report to describe the algorithm you have employed)

b) evaluate the accuracy of results.

Your code should employ a "batch means" technique that adaptively chooses  the number of batches so to achieve outputs with a desired degree of accuracy. 

Define properly the accuracy metric, which should be related to the width of confidence intervals.

Plot of the average delay in function of the utilisation, where the utilisation is: 0.1, 0.2, 0.4, 0.7, 0.8, 0.9, 0.95, 0.99. Show also the 95%-level confidence intervals.

Consider three scenarios for the service time:

EXP: exponentially distributed with mean=1

DET: deterministic =1

HYP:  distributed according to a hyper-exponential distribution with mean=1 standard deviation=10

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as ss
import random
import numpy as np

# Defining functions and classes

In [ ]:
def arrival(time, FES, queue, average_arrival_time, average_service_time):
    
    global users
    global customer
    
    # introducing random client arrival
    # inter_arrival = np.random.exponential(1.0/average_arrival_time)
    inter_arrival = random.expovariate(1.0/average_arrival_time)
    #FES.put((time + inter_arrival, 'arrival'))
    FES.append((time + inter_arrival, 'arrival'))
    
    # managing the event 
    users += 1
    x = 'client' + str(customer)
    customer += 1
    
    # recording client id and put it in the list
    client = Client(x, time)
    queue.append(client)

    print(f'{client.name} arrived at {client.arrival_time}')
    
    # start the service in case the server is idle
    if users == 1:
        # scheduling random departure time to the clients
        # service_time = np.random.exponential(1.0/average_service_time)
        # service_time = random.expovariate(1.0/average_service_time)
        service_time = average_service_time
        FES.append((time + service_time, 'departure'))


def departure(time, FES, queue, average_arrival_time, average_service_time):
    
    global users
    
    # manipulating the list of clients to get FIFO orientation
    queue.reverse()
    client = queue.pop()
    queue.reverse()
    users -= 1
    delay = time - client.arrival_time
    
    print(f'{client.name} departured at {time}')
    
    # checking the number of clients in line
    if users > 0:
        # scheduling random departure time to the clients
        # service_time = np.random.exponential(1.0/average_service_time)
        # service_time = random.expovariate(1.0/average_service_time)
        service_time = average_service_time
        FES.append((time + service_time, 'departure'))
    
    return delay

class Client:
    def __init__(self, name, arrival_time):
        self.name = name
        self.arrival_time = arrival_time
        

# Finding transient knee (k)
In order to find the transient knee (k), we should implement the simulation for a long time and compute the average. Afterwards, we should calculate the average depending on k to plot. Fianlly, k is determined by this plot.

In [ ]:
def finding_transient(delay_list):
    
    delay_ave = sum(delay_list) / len(delay_list)
    delay_ave_k = {}
    delay_ave_r = {}
    
    for k in range(len(delay_list)):
        temp = sum(delay_list[(k):]) / (len(delay_list) - k)
        delay_ave_k[k] = temp
        delay_ave_r[k] = (temp/delay_ave) - 1
    
    print('*****************************', k)
    plt.plot(delay_ave_k.keys(), delay_ave_k.values())
    # plt.plot(delay_ave_r.keys(), delay_ave_r.values())
    
    return delay_ave_k

In [ ]:
# def transient_point(variance_list):
    
#     var_ave = np.mean(variance_list)
#     var_std = np.std(variance_list)
#     print(var_ave, var_std)
#     var_max = max(variance_list)
#     j = variance_list.index(var_max)
    
#     for i in range(j, len(variance_list)):
#         if (variance_list[i] > var_ave - var_std/2) and (variance_list[i] < var_ave + var_std/2):
#             return i


def transient_point(variance_list, u):
    
    var_ave = np.mean(variance_list)
    var_std = np.std(variance_list)
    
    if u <= 0.7:
        j = int(len(variance_list) * u)
    else:
        j = int(len(variance_list) * 0.7)
    
    for i in range(j, len(variance_list)):
        if (variance_list[i] > var_ave - var_std) and (variance_list[i] < var_ave + var_std):
            return i



def confidence_interval_margin(batch_mean_list, confidence_interval):
    
    mu = np.mean(batch_mean_list)
    std = np.std(batch_mean_list)
    n = len(batch_mean_list)
    t = np.abs(ss.t.ppf((1-confidence_interval)/2, n - 1))
    
    margin = t * std / np.sqrt(n)
    
    expanding_condition = 2 * margin / mu
    
    return margin, expanding_condition

In [ ]:
np.random.seed(32)
random.seed(42)

simulation_time = 50000
simulation_time_warm_up = 20000
utilization = [0.1, 0.2, 0.4, 0.7, 0.8, 0.9, 0.95, 0.99]
distributions = ['deterministic', 'exponential', 'hyperexponential']
# average_service_time = 1
cumulative_delay = []
variance_delay = []

accuracy = 0.016

for u in [0.99]:
    # initialization of variables
    
    time = 0
    users = 0
    customer = 1
    queue = []
    FES = []
    delay = []
    
    confidence_interval = 0.95
    
    batch_mean_list = []
    expanding_number = 0
    batche_initial_size = 10
    batch_count = 10
    flag =1
    
    FES.append((0,'arrival'))
    target = 0
    
    # Lambda = u/average_service_time
    # average_arrival_time = 1/Lambda
    
    while time < simulation_time and flag:
        average_service_time = random.expovariate(1)
        Lambda = u/average_service_time
        average_arrival_time = 1/Lambda 

        
        FES = sorted(FES)
        (time, event_type) = FES[target]
        
        if event_type == 'arrival':
            arrival(time, FES, queue, average_arrival_time, average_service_time)
        elif event_type == 'departure':
            delay.append(departure(time, FES, queue, average_arrival_time, average_service_time))
            cumulative_delay.append(sum(delay)/len(delay))
            variance_delay.append(np.var(delay))
        
        if time > (simulation_time_warm_up + expanding_number):
            
            if expanding_number == 0:
                k = transient_point(variance_delay, u)
                # batch_size = int(((len(cumulative_delay) - k)/batche_initial_size))
                batch_size = int(((simulation_time_warm_up - k)/batche_initial_size))
                batch_start_index = k
            else:
                batch_start_index = len(cumulative_delay) - batch_size
            
            
            while(batch_start_index < len(cumulative_delay)):
                batch_mean_list.append(np.mean(cumulative_delay[batch_start_index:(batch_start_index + batch_size)]))
                batch_start_index += batch_size
            
        
            margin, expanding_condition  = confidence_interval_margin(batch_mean_list, confidence_interval)
            print(f'margin is:*******************************{margin}**********************')
            
            if expanding_condition > accuracy:
                expanding_number += batch_size
                batch_count += 1
            else:
                flag = 0 
            
            # if expanding_number == 0:
            #     P = margin * u
            #     expanding_number += batch_size
            #     batch_count += 1
            #     print(f'P is:*******************************{P}**********************')
            # elif expanding_condition > P:
            #     expanding_number += batch_size
            #     batch_count += 1
            # else:
            #     flag = 0 
                
        
            
        target += 1
        
    # # temp = finding_transient(delay)
    # plt.figure()
    # plt.subplot(121)
    # plt.plot(cumulative_delay)
    # plt.subplot(122)
    # plt.plot(cumulative_delay)
    # plt.show()
    # k = transient_point(cumulative_delay)
    # print(f'warm_up point for utilization{u} is {k}.')     
        

In [ ]:
from scipy.interpolate import make_interp_spline
x = range(len(variance_delay))
y = variance_delay
xyspl = make_interp_spline(x, y)
x_ = np.linspace(min(x), max(x), 5000)
y_ = xyspl(x_)

# plt.plot(len(variance_delay), cumulative_delay)
plt.plot(x_, y_, label='kir')
#plt.yscale('log')
plt.legend()

In [ ]:
plt.plot(cumulative_delay)
# plt.yscale('log')
# plt.plot(variance_delay, label='kir')
# plt.yscale('log')
plt.legend()

In [ ]:
plt.plot(cumulative_delay)
# plt.plot(variance_delay, label='kir')
# plt.yscale('log')
plt.legend()

In [ ]:
mu = np.mean(cumulative_delay[4000:])
std = np.std(cumulative_delay[4000:])
# mu = cumulative_delay[-1]
# std = np.sqrt(variance_delay[-1])

hasan = 2*1.96*std/(mu*200)
hasan, mu, std, variance_delay[-1]

In [ ]:
t = np.abs(ss.t.ppf((0.05), 2))
t

In [ ]:
batch_count